In [1]:
%reload_ext autoreload
%autoreload 2

# Use aligner_v7 kernel

import sys
add_paths = [
    "/fsx_0/user/tranx/rsync", # ALIGNER_PARENT_DIR
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/replicated", # ALIGNER_PARENT_DIR/llm_mm_aligner/replicated
    # "/data/home/tranx/conda/envs/aligner_20240822_v2/python-packages", #CONDA_PREFIX/python-packages
    # "/data/home/kapilk/.conda/envs/aligner_20240822_v2/python-packages"
    "/fsx_0/shared/conda/aligner_20241030/python-packages"
]

for p in add_paths:
    if p not in sys.path:
        sys.path.append(p)
        
# device = "cuda:2"

In [3]:
class PerceiverV3Tokenizer(PerceptionTokenizer):
    def get_model(self) -> nn.Module:
        num_heads = self.model_args.perceiver_num_heads
        if self.model_args.perceiver_dim_override:
            dim = self.model_args.perceiver_dim_override
        else:
            dim = self.feature_dim

        mult = num_heads * 64  # dim head should be a multiple of 64
        dim = int(ceil(dim / mult)) * mult

        return PerceiverResamplerV3(
            in_dim=self.feature_dim,
            dim=dim,
            out_dim=self.embedding_size,
            depth=self.model_args.perception_tokenizer_num_layers,
            ff_mult=self.model_args.perceiver_ff_mult,
            num_latents=self.model_args.perceiver_num_latents,
            heads=num_heads,
            kv_heads=self.model_args.perceiver_num_kv_heads,
            hidden_dropout_p=self.model_args.perception_tokenizer_hidden_dropout_p,
            attention_dropout_p=self.model_args.perception_tokenizer_attention_dropout_p,
            num_img_chunks=self.data_args.num_image_chunks,
            cat_latents=self.model_args.perceiver_cat_latents,
            collapse_chunks=self.model_args.perceiver_collapse_chunks,
            enable_query_aware=self.model_args.perceiver_enable_query_aware,
            enable_moe=self.model_args.perceiver_enable_moe,
            moe_num_experts=self.model_args.perceiver_num_experts,
            moe_num_activated_experts=self.model_args.perceiver_num_activated_experts,
            add_output_norm=self.model_args.perceiver_add_output_norm,
            enforce_uniform_emb_variance=self.model_args.enforce_uniform_emb_variance,
            use_temporal_position_encoding=self.model_args.perceiver_temporal_position_encoding,
        )

NameError: name 'PerceptionTokenizer' is not defined

In [12]:
from llm_mm_aligner.lib.tokenizers.perceiverIO_projector import PerceiverResamplerV3
from math import ceil
import numpy as np

In [ ]:
dim = 4096
num_heads = 32
num_layers = 22

# in_dim = 4096
# in_dim = dim
# mult = num_heads * 64
# dim = int(ceil(dim / mult)) * mult

# for num_layers in [14,18,22,26,30]:
for dim in [2048, 3072, 5120, 6144]:
    model = PerceiverResamplerV3(
        in_dim=dim,
        dim=dim,
        out_dim=dim,
        depth=num_layers,
        heads=num_heads,
        kv_heads=8, # per job config, true for both 8B and 70B
    )

    # self.model_args.perceiver_num_kv_heads: default = 16

    # https://fburl.com/code/29v9guvf
    # self.to_q = nn.Linear(self.dim, self.dim_head * self.heads, bias=False)
    # self.to_k = nn.Linear(self.dim, self.dim_head * self.kv_heads, bias=False)
    # self.to_v = nn.Linear(self.dim, self.dim_head * self.kv_heads, bias=False)

    # model
    print("-"*50)
    print(f"{dim=}, {num_heads=}, {num_layers=}")
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model size (B) = {np.round(total_params/1e9, 2)}")

--------------------------------------------------
in_dim=4096, mult=2048, dim=2048, num_layers=22
Model size (B) = 1.34
--------------------------------------------------
in_dim=4096, mult=2048, dim=3072, num_layers=22
Model size (B) = 2.18
--------------------------------------------------
in_dim=4096, mult=2048, dim=5120, num_layers=22
Model size (B) = 6.98
--------------------------------------------------
in_dim=4096, mult=2048, dim=6144, num_layers=22
Model size (B) = 8.72
